# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

openai = OpenAI()

In [ ]:
# read the user question
def user_question_reader() -> str:
    input_text = "Hello! I’m your AI Teacher, ready to help you explore any topic you’re curious about.\n"
    input_text +="I have access to a vast amount of knowledge and will do my best to explain things clearly, no matter your experience level.\n\n"
    input_text +="What would you like to learn about today?\n"

    question = input(input_text)

    return question;

In [ ]:
# generate the user prompt
def user_prompt_generator() -> str:
    question = user_question_reader()
    user_prompt = f"I need you to answer to this question: {question}.\n"
    user_prompt += " Take into account that I dont have prior knowledge about my question \
    so I want the answer as complete as possible. Also please provide it in markdown"

    return user_prompt

In [ ]:
# define the system prompt
system_prompt = "Your job it's to be a teacher. You have access to all the knowledge \
                in the internet. You will be thankful to any question given to you and \
                will try to answer it the best you can. Your students might know little to nothing \
                about what they ask and make mistakes so you will have to think about the meaning of their question \
                before and provide an answer according to the meaning behind it."

In [ ]:
# define gpt-4o-mini function to answer, with streaming
def gpt_teacher():
    stream = openai.chat.completions.create(
        model = MODEL_GPT,
        messages= [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content" : user_prompt_generator()}
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# define Llama 3.2 function to answer
def llama_teacher():
    response = ollama.chat(
        model = MODEL_LLAMA,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role": "user", "content": user_prompt_generator()}
        ]
    )

    return display(Markdown(response['message']['content']))

In [ ]:
# try the gpt teacher
gpt_teacher()

In [ ]:
#try the ollama teacher
llama_teacher()